In [1]:
in_funds = ['VFINX', 'PRIDX']
out_funds = ['VUSTX']

In [2]:
import yfinance as yf

# Download historical data
data = {}
all_funds = in_funds + out_funds
for symbol in all_funds:
    print(symbol)
    yf_ticker = yf.Ticker(symbol)
    yf_data = yf_ticker.history(period='max')
    data[symbol] = yf_data

VFINX
PRIDX
VUSTX


In [3]:
# Select only month-end prices
monthend = {}
for symbol, df in data.items():
    day1 = list(df.index.month)
    day2 = list(df.index.month)
    day2.pop(0)
    day2.append(-1)
    days = zip(day1, day2)
    truth_vals = [False if x[0] == x[1] else True for x in days]
    df_month_end = df[truth_vals].copy()
    monthend[symbol] = df_month_end

In [4]:
# Calculate momentum signals
import numpy

# 1-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum1 = (((close[1:]/close[:-1])-1)*100)
    momentum1 = numpy.insert(momentum1, 0, 0)
    df['Momentum1'] = momentum1

# 3-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum3 = (((close[3:]/close[:-3])-1)*100)
    momentum3 = numpy.insert(momentum3, 0, [0, 0, 0])
    df['Momentum3'] = momentum3

# 6-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum6 = (((close[6:]/close[:-6])-1)*100)
    momentum6 = numpy.insert(momentum6, 0, [0, 0, 0, 0, 0, 0])
    df['Momentum6'] = momentum6

# ADM Signal
for symbol, df in monthend.items():
    momentums = numpy.array((df['Momentum1'], df['Momentum3'], df['Momentum6']))
    df['ADMSignal'] = numpy.average(momentums, axis=0)

In [5]:
# Align time-axis for all symbols
import pandas
merged = None
for symbol, df in monthend.items():
    if not symbol in in_funds:
        continue
    if merged is None:
        merged = pandas.DataFrame(df['ADMSignal'])
        merged[f'{symbol}-ADMSignal'] = merged['ADMSignal']
        merged = merged.drop(columns=['ADMSignal'])
    else:
        df2 = pandas.DataFrame(df['ADMSignal'])
        df2[f'{symbol}-ADMSignal'] = df2['ADMSignal']
        df2 = df2.drop(columns=['ADMSignal'])
        merged = merged.merge(df2, how='inner', on='Date')
time_aligned = merged
print(time_aligned)

            VFINX-ADMSignal  PRIDX-ADMSignal
Date                                        
1988-12-30         2.696485         0.000000
1989-01-31         8.745183         1.393035
1989-02-28         5.477646         0.095511
1989-03-31         6.517398         1.297797
1989-04-28         7.655554         2.483286
...                     ...              ...
2020-03-31       -14.820500       -16.504324
2020-04-30         0.075828         0.524241
2020-05-29         2.025616         6.813656
2020-06-30         6.426191        13.170344
2020-07-08         3.524771        10.583333

[380 rows x 2 columns]


In [8]:
# Build portfolio
symbols = pandas.DataFrame(time_aligned.idxmax(axis=1), columns=['Symbol'])
symbols['Symbol'] = [x.split('-')[0] for x in symbols['Symbol']]
signal  = pandas.DataFrame(time_aligned.max(axis=1), columns=['Signal'])
in_signal = symbols.merge(signal, how='inner', on='Date')
for idx in in_signal[in_signal['Signal'] < 0].index:
    in_signal.loc[idx, 'Symbol'] = 'VUSTX'
portfolio = in_signal

In [9]:
# Print out last few items, to show what should be purchased
portfolio.tail()

,Symbol,Signal
Date,,
2020-03-31,VUSTX,-14.820500
2020-04-30,PRIDX,0.524241
2020-05-29,PRIDX,6.813656
2020-06-30,PRIDX,13.170344
2020-07-08,PRIDX,10.583333
